In [1]:
import pandas as pd

In [2]:
!pip install transformers datasets seqeval


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16183 sha256=395af15d8c162761d604764ff877b97ffdcfd140ff9e92c063a492ee502af8a8
  Stored in directory: c:\users\yibabe\appdata\local\pip\cache\wheels\5f\b8\73\0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval


In [3]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_name = "xlm-roberta-base"  # You can also use bert-tiny-amharic if available
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=5)  # Adjust num_labels based on your entity types (e.g., products, prices, locations)


c:\Users\Yibabe\Desktop\10academyAIMweek-5\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Yibabe\Desktop\10academyAIMweek-5\env\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Error while downloading from https://cdn-lfs.hf.co/xlm-roberta-base/6fd4797bc397c3b8b55d6bb5740366b57e6a3ce91c04c77f22aafc0c128e6feb?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1727892797&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbi

In [7]:
from datasets import load_dataset

# Load the dataset with `trust_remote_code=True`
dataset = load_dataset("conll2003", split="train", trust_remote_code=True)

# Check the first few rows to ensure it's loaded correctly
print(dataset[0])


Generating test split: 100%|██████████| 3453/3453 [00:01<00:00, 3179.55 examples/s]


{'id': '0', 'tokens': ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7], 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0], 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}


In [8]:
from transformers import AutoTokenizer

# Load the XLM-Roberta tokenizer
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")


c:\Users\Yibabe\Desktop\10academyAIMweek-5\env\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [16]:

# Load the entire data from the CSV file
data_file_path = r'C:\Users\Yibabe\Desktop\10academyAIMweek-5\data\cleaned_tokenized_and_labled_data.csv'  # Update this to the correct path of your CSV file
df = pd.read_csv(data_file_path)


In [17]:
df.head()

,Labeled_Message
0,Car B-PRODUCT\nAromatherapy I-PRODUCT\nSolar I...
1,Car B-PRODUCT\nAromatherapy I-PRODUCT\nSolar I...
2,GW O\nHAIR O\nDRYER/Blower O\nየፀጉር O\nማድረቂያ O\...
3,2 B-PRODUCT\nin I-PRODUCT\n1 I-PRODUCT\nPorcel...
4,Plastic B-PRODUCT\nAnd I-PRODUCT\nMetal I-PROD...


In [20]:

# Assuming the relevant text column is named 'Labeled_Message'; adjust this if needed
if 'Labeled_Message' not in df.columns:
    raise ValueError("The 'Labeled_Message' column is not found in the DataFrame.")

# Redundant messages to be removed
redundant_message = [
    "ከፍለው O",
    "ካሉበት O",
    "እንልካለን O",
    "ክፍለሃገር O",
    "ላላችሁ O",
    "ደንበኞቻችን O",
    "የፈለጉትን O",
    "ዕቃ O",
    "በመልዕክት O",
    "እንልክልዎታለን O",
    "ለማዘዝ O",
    "@ordermertteka1 O",
    "@ordermertteka2 O",
    "ለወዳጅዎ O",
    "forward O",
    "በማድረግ O",
    "ይተባበሩን O",
    "0944-22-23-24 O",
    "0904-94-48-48 O",
    "አድራሻችን O",
    "መገናኛ I-LOC",
    "ዘፍመሽ O",
    "ግራንድ O",
    "ሞል O",
    "3ኛ O",
    "ፎቅ O",
    "ከሊፍት O",
    "ሲወርዱ O",
    "ወደ O",
    "ቀኝ O",
    "ታጥፈው O",
    "ቀጥታ O",
    "376 I-LOC",
    "በኪስዎ O",
    "ጥሬ O",
    "ገንዘብ O",
    "ካልያዙ O",
    "በሞባይል O",
    "ማስተላለፍ O",
    "ይችላሉ። O",
    "ይሄንን O",
    "t.me/MerttEka O",
    "ተጭነው O",
    "join O",
    "ያድርጉ፣ O",
    "ቤተሰብ O",
    "ይሁኑ O"
]

# Function to clean redundant messages
def clean_redundant_messages(text):
    # Check if the text is a string
    if isinstance(text, str):
        for message in set(redundant_message):
            text = text.replace(message + "\n", "")
        return text.strip()  # Remove leading/trailing whitespace after replacement
    return None  # Return None for non-string entries

# Clean the redundant messages in the DataFrame
df['cleaned_text'] = df['Labeled_Message'].apply(clean_redundant_messages)

# Function to split tokens and labels
def split_tokens_and_labels(text):
    if isinstance(text, str):  # Ensure the input is a string
        lines = text.strip().split('\n')
        tokens = []
        labels = []
        for line in lines:
            if line.strip():  # Ensure the line is not empty
                token, label = line.rsplit(' ', 1)
                tokens.append(token)
                labels.append(label)
        return tokens, labels
    else:
        return [], []  # Return empty lists for non-string entries

# Apply the token splitting
df['tokens_labels'] = df['cleaned_text'].apply(split_tokens_and_labels)

# Separate tokens and labels into two new columns
df['tokens'] = df['tokens_labels'].apply(lambda x: x[0])
df['labels'] = df['tokens_labels'].apply(lambda x: x[1])

# Save the cleaned DataFrame to a new CSV file
output_file_path = r'C:\Users\Yibabe\Desktop\10academyAIMweek-5\data\cleaned_data.csv' # Update this to your desired output path
df.to_csv(output_file_path, index=False)




In [21]:
# Display the resulting DataFrame (optional)
df[['cleaned_text', 'tokens', 'labels']]

,cleaned_text,tokens,labels
0,Car B-PRODUCT\nAromatherapy I-PRODUCT\nSolar I...,"[Car, Aromatherapy, Solar, Vortex, ይሁኑ]","[B-PRODUCT, I-PRODUCT, I-PRODUCT, I-PRODUCT, O]"
1,Car B-PRODUCT\nAromatherapy I-PRODUCT\nSolar I...,"[Car, Aromatherapy, Solar, Vortex, የመኪና, መዓዛ, ...","[B-PRODUCT, I-PRODUCT, I-PRODUCT, I-PRODUCT, O..."
2,GW O\nHAIR O\nDRYER/Blower O\nየፀጉር O\nማድረቂያ O\...,"[GW, HAIR, DRYER/Blower, የፀጉር, ማድረቂያ, ፎን, 6000...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,2 B-PRODUCT\nin I-PRODUCT\n1 I-PRODUCT\nPorcel...,"[2, in, 1, Porcelain, Dessert, Bowel, የሰላጣ, እና...","[B-PRODUCT, I-PRODUCT, I-PRODUCT, I-PRODUCT, I..."
4,Plastic B-PRODUCT\nAnd I-PRODUCT\nMetal I-PROD...,"[Plastic, And, Metal, Cubic, Cloth, Cabinet, ዘ...","[B-PRODUCT, I-PRODUCT, I-PRODUCT, I-PRODUCT, I..."
...,...,...,...
4078,2500 B-PRODUCT,[2500],[B-PRODUCT]
4079,ዋጋ B-PRODUCT\n2500 I-PRODUCT\n0983063957 O,"[ዋጋ, 2500, 0983063957]","[B-PRODUCT, I-PRODUCT, O]"
4080,የሙያ B-PRODUCT\nባለቤት I-PRODUCT\nመሆን I-PRODUCT\n...,"[የሙያ, ባለቤት, መሆን, መሠልጠን, ነው።, ቀለም, ቀቢ, ሳያስፈልግዎ,...","[B-PRODUCT, I-PRODUCT, I-PRODUCT, I-PRODUCT, I..."
4081,ቤትና B-PRODUCT\nግቢዎን I-PRODUCT\nእንዲሁም I-PRODUCT...,"[ቤትና, ግቢዎን, እንዲሁም, የብረት, እና, የእንጨት, ቁሳቁስዎን, ቀለ...","[B-PRODUCT, I-PRODUCT, I-PRODUCT, I-PRODUCT, I..."


In [33]:

# Function to split cleaned text into tokens and labels
def split_tokens_labels(row):
    cleaned_text = row['cleaned_text']
    
    # Check if cleaned_text is None or an empty string
    if cleaned_text is None or cleaned_text.strip() == "":
        return ([], []), [], []  # Return empty lists if cleaned_text is not valid
    
    # Split the cleaned text into lines
    lines = cleaned_text.strip().split('\n')
    
    # Initialize lists for tokens and labels
    tokens = []
    labels = []
    
    # Iterate through each line
    for line in lines:
        # Split each line by space
        parts = line.rsplit(' ', 1)  # Split only at the last space
        if len(parts) == 2:  # Ensure there's a token and a label
            token, label = parts
            tokens.append(token)
            labels.append(label)
    
    # Create tokens_labels format as a tuple of lists
    tokens_labels = (tokens, labels)
    
    return tokens_labels, tokens, labels

# Apply the function to the DataFrame
df[['tokens_labels', 'tokens', 'labels']] = df.apply(split_tokens_labels, axis=1, result_type='expand')



In [34]:

# Display the updated DataFrame
df[['cleaned_text', 'tokens_labels', 'tokens', 'labels']].head()

,cleaned_text,tokens_labels,tokens,labels
0,Car B-PRODUCT\nAromatherapy I-PRODUCT\nSolar I...,"([Car, Aromatherapy, Solar, Vortex, ይሁኑ], [B-P...","[Car, Aromatherapy, Solar, Vortex, ይሁኑ]","[B-PRODUCT, I-PRODUCT, I-PRODUCT, I-PRODUCT, O]"
1,Car B-PRODUCT\nAromatherapy I-PRODUCT\nSolar I...,"([Car, Aromatherapy, Solar, Vortex, የመኪና, መዓዛ,...","[Car, Aromatherapy, Solar, Vortex, የመኪና, መዓዛ, ...","[B-PRODUCT, I-PRODUCT, I-PRODUCT, I-PRODUCT, O..."
2,GW O\nHAIR O\nDRYER/Blower O\nየፀጉር O\nማድረቂያ O\...,"([GW, HAIR, DRYER/Blower, የፀጉር, ማድረቂያ, ፎን, 600...","[GW, HAIR, DRYER/Blower, የፀጉር, ማድረቂያ, ፎን, 6000...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,2 B-PRODUCT\nin I-PRODUCT\n1 I-PRODUCT\nPorcel...,"([2, in, 1, Porcelain, Dessert, Bowel, የሰላጣ, እ...","[2, in, 1, Porcelain, Dessert, Bowel, የሰላጣ, እና...","[B-PRODUCT, I-PRODUCT, I-PRODUCT, I-PRODUCT, I..."
4,Plastic B-PRODUCT\nAnd I-PRODUCT\nMetal I-PROD...,"([Plastic, And, Metal, Cubic, Cloth, Cabinet, ...","[Plastic, And, Metal, Cubic, Cloth, Cabinet, ዘ...","[B-PRODUCT, I-PRODUCT, I-PRODUCT, I-PRODUCT, I..."


In [35]:
# Create a dataset for training
train_data = [(row['tokens'], row['labels']) for _, row in df.iterrows() if row['tokens'] and row['labels']]


In [36]:
from collections import Counter

# Flatten labels from the train_data
all_labels = [label for _, labels in train_data for label in labels]
label_counts = Counter(all_labels)

print(label_counts)  # Display the counts of each label


Counter({'O': 82396, 'I-PRODUCT': 14789, 'I-PRICE': 11316, 'B-PRODUCT': 4066, 'I-LOC': 2140, 'o': 1, 'price': 1})
